###### NAME: JANMEJAY MOHANTY <br> COURSE: CS 583 A <br> COURSE NAME: DEEP LEARNING <br> CWID: 20009315 <br> EMAIL: JMOHANTY@STEVENS.EDU

In [806]:
import pandas as pd
import numpy as np
import random
import keras.backend as K
K.clear_session()
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [807]:
#Please make sure that both h3data folder and this ipython notebook are in same path/folder
!ls | grep "h3data"

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [808]:
data = pd.read_csv('./h3data/data.txt', sep=" ", header=None)
data.columns = ["Input", "output"]
data.head()

,Input,output
0,apprehensive,apprehensiveway
1,desperately,esperatelyday
2,grew,ewgray
3,niggardly,iggardlynay
4,brought,oughtbray


In [809]:
#You can consider each character as a word and train a RNN model to predict the output for a given input
data["Input"] = data["Input"].astype(str).apply(lambda x:" ".join(list(x)))
data["output"] = data["output"].astype(str).apply(lambda x:" ".join(list(x)))
data.head()

,Input,output
0,a p p r e h e n s i v e,a p p r e h e n s i v e w a y
1,d e s p e r a t e l y,e s p e r a t e l y d a y
2,g r e w,e w g r a y
3,n i g g a r d l y,i g g a r d l y n a y
4,b r o u g h t,o u g h t b r a y


In [810]:
data.columns = ("Encoder Input", "Output")
data["Decoder Input"] = data["Output"].apply(lambda x: "<start> "+x[:-1])
data.head()

,Encoder Input,Output,Decoder Input
0,a p p r e h e n s i v e,a p p r e h e n s i v e w a y,<start> a p p r e h e n s i v e w a
1,d e s p e r a t e l y,e s p e r a t e l y d a y,<start> e s p e r a t e l y d a
2,g r e w,e w g r a y,<start> e w g r a
3,n i g g a r d l y,i g g a r d l y n a y,<start> i g g a r d l y n a
4,b r o u g h t,o u g h t b r a y,<start> o u g h t b r a


In [811]:
print("Total number of data points: ",data.shape[0])

Total number of data points:  6511


Split the data.txt into 80% training and 20% and get the accuracy on the test set. [2 Points]


In [812]:
#fill the code to split the data into 80% train and 20% test data
random.seed(100)
train_index = np.random.choice(data.shape[0]-1,round((data.shape[0]-1)*0.8),replace = False)
test_index = [idx for idx in data.index if idx not in train_index]
train = data.iloc[train_index,:]
test = data.iloc[test_index,:]
print(train.shape)
print(test.shape)

(5208, 3)
(1303, 3)


In [813]:
from collections import defaultdict

vocab_list = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z","-","'","`","<start>"]

vocab = defaultdict(lambda : 0)
count = 1


for i in vocab_list:
  if vocab[i]==0:
    vocab[i] = count
    count = count + 1

In [814]:
len(vocab)

30

In [815]:
X_train_encoder = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in train["Encoder Input"].values]
X_train_decoder = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in train["Decoder Input"].values]
Y_train         = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in train["Output"].values]

X_test_encoder = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in test["Encoder Input"].values]
X_test_decoder = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in test["Decoder Input"].values]
Y_test         = [ [vocab[char] for char in x.split() if vocab[char] !=0] for x in test["Output"].values]

In [816]:
X_train_encoder[0]

[13, 9, 19, 3, 8, 9, 5, 22, 15, 21, 19]

In [817]:
X_train_decoder[0]

[30, 9, 19, 3, 8, 9, 5, 22, 15, 21, 19, 13, 1]

In [818]:
Y_train[0]

[9, 19, 3, 8, 9, 5, 22, 15, 21, 19, 13, 1, 25]

In [819]:
X_train_encoder = pad_sequences(X_train_encoder, maxlen=15)
X_train_decoder = pad_sequences(X_train_decoder, maxlen=15)
Y_train = pad_sequences(Y_train, maxlen=15)

X_test_encoder = pad_sequences(X_test_encoder, maxlen=15)
X_test_decoder = pad_sequences(X_test_decoder, maxlen=15)
Y_test = pad_sequences(Y_test, maxlen=15)

In [820]:
# Define input and output layers
input_layer = Input(shape=(None,))
output_layer = Input(shape=(None,))

# Define the encoder RNN
encoder_embedding = Embedding(input_dim=31, output_dim=10)(input_layer)
encoder_rnn, state_h, state_c = LSTM(units=64, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder RNN
decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=31, output_dim=10)(decoder_input)
decoder_rnn = LSTM(units=64, return_sequences=True)(decoder_embedding, initial_state=encoder_states)
decoder_output = Dense(units=31, activation='softmax')(decoder_rnn)

# Define the model
model = Model(inputs=[input_layer, decoder_input], outputs=decoder_output)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 10)     310         ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 10)     310         ['input_3[0][0]']                
                                                                                              

The model is trained via a cross-entropy loss between the decoder distribution 
and ground-truth at each time step. [2 Points]

In [821]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [822]:
model.fit([X_train_encoder,X_train_decoder],Y_train,batch_size=16,epochs=30)

Epoch 1/30
326/326 [==============================] - 6s 10ms/step - loss: 1.8673 - accuracy: 0.4856
Epoch 2/30
326/326 [==============================] - 4s 11ms/step - loss: 1.3493 - accuracy: 0.5974
Epoch 3/30
326/326 [==============================] - 4s 11ms/step - loss: 1.2380 - accuracy: 0.6297
Epoch 4/30
326/326 [==============================] - 3s 10ms/step - loss: 1.1675 - accuracy: 0.6508
Epoch 5/30
326/326 [==============================] - 4s 11ms/step - loss: 1.1116 - accuracy: 0.6691
Epoch 6/30
326/326 [==============================] - 4s 12ms/step - loss: 1.0720 - accuracy: 0.6804
Epoch 7/30
326/326 [==============================] - 4s 12ms/step - loss: 1.0305 - accuracy: 0.6939
Epoch 8/30
326/326 [==============================] - 4s 12ms/step - loss: 0.9914 - accuracy: 0.7040
Epoch 9/30
326/326 [==============================] - 4s 13ms/step - loss: 0.9594 - accuracy: 0.7102
Epoch 10/30
326/326 [==============================] - 4s 13ms/step - loss: 0.9273 - accura

In [823]:
model.save('lstm_model.h5')

In [824]:
loss, accuracy = model.evaluate(x=[X_test_encoder,X_test_decoder], y=Y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

41/41 [==============================] - 1s 4ms/step - loss: 0.4459 - accuracy: 0.8655
Test loss: 0.44588130712509155
Test accuracy: 0.8655410408973694
